In [1]:
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import find_dotenv, load_dotenv
load_dotenv(find_dotenv())

gemini_api_key = os.getenv("GEMINI_API_KEY")

llm = ChatGoogleGenerativeAI(model="gemini-pro", google_api_key=gemini_api_key, temperature=0)

/home/cayena/.cache/pypoetry/virtualenvs/langchain-examples-NuBMPSch-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## CSV Parser

Esse tipo de parser pode ser utilizado quando você quiser devolver a saída de uma LLM no formato csv.

In [4]:
from langchain.output_parsers import CommaSeparatedListOutputParser

output_parser = CommaSeparatedListOutputParser()
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

Your response should be a list of comma separated values, eg: `foo, bar, baz`


In [8]:
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

prompt = PromptTemplate(
    template="Liste 5 fatos sobre o assunto: {assunto}.\n{format_instructions}",
    input_variables=["assunto"],
    partial_variables={"format_instructions": format_instructions},
)
print(prompt.format(assunto="Maiores pontuadores da NBA"))

Liste 5 fatos sobre o assunto: Maiores pontuadores da NBA.
Your response should be a list of comma separated values, eg: `foo, bar, baz`


In [9]:
chain = prompt | llm | output_parser

In [11]:
resp_llm = chain.invoke({"assunto": "Morar no Brasil"})

In [15]:
for curiosidade in resp_llm:
    print(curiosidade)

* Custo de vida relativamente baixo em comparação com outros países desenvolvidos
* Clima tropical com praias deslumbrantes e florestas exuberantes
* Cultura vibrante com música
dança e culinária diversificadas
* Desigualdade social e violência são desafios significativos
* Economia em desenvolvimento com oportunidades crescentes


## Datetime Parser

Este parser é utilizado quando você quer devolver a saída de uma LLM no formato de data e hora.

In [20]:
from langchain.output_parsers import DatetimeOutputParser

datatime_parser = DatetimeOutputParser()
format_instructions = datatime_parser.get_format_instructions()
print(format_instructions)

Write a datetime string that matches the following pattern: '%Y-%m-%dT%H:%M:%S.%fZ'.

Examples: 1047-11-12T16:34:13.715542Z, 387-08-21T23:09:20.231583Z, 1652-07-17T04:49:58.529305Z

Return ONLY this string, no other words!


In [22]:
template = """Responda a pergunta abaixo:

{pergunta}

{format_instructions}"""

prompt = PromptTemplate(
    template=template,
    partial_variables={"format_instructions": format_instructions},
    input_variables=["pergunta"]
)
print(prompt.format(pergunta="Quando o Facebook foi fundado?"))

Responda a pergunta abaixo:

Quando o Facebook foi fundado?

Write a datetime string that matches the following pattern: '%Y-%m-%dT%H:%M:%S.%fZ'.

Examples: 1047-11-12T16:34:13.715542Z, 387-08-21T23:09:20.231583Z, 1652-07-17T04:49:58.529305Z

Return ONLY this string, no other words!


In [23]:
chain = prompt | llm | datatime_parser

In [25]:
resp_llm = chain.invoke(
    {"pergunta": "Quando o Facebook foi fundado?"}
)

In [26]:
print(resp_llm)

2004-02-04 19:00:00


## JSON Parser

Este parser é utilizado quando você quer devolver a saída de uma LLM no formato JSON.

Para este parser, você necessita de uma LLM que traga respostas confiável em JSON. 
Alguns modelos pode não trazer corretamente a estrutura e dar problemas na execução. 
Para isso, usamos o pydantic para validar a estrutura do JSON.

In [28]:
from langchain_core.pydantic_v1 import BaseModel, Field

class NER(BaseModel):
    pergunta: str = Field(description="Sentença do usuário")
    nome: str = Field(description="Nome de alguma pessoa citada na sentença. Se tiver mais nomes, inserir como lista")
    empresa: str = Field(description="Nome da empresa que tal pessoa trabalha. Se tiver mais de uma, inserir como lista")

In [30]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser(pydantic_object=NER)
print(json_parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"pergunta": {"title": "Pergunta", "description": "Senten\u00e7a do usu\u00e1rio", "type": "string"}, "nome": {"title": "Nome", "description": "Nome de alguma pessoa citada na senten\u00e7a. Se tiver mais nomes, inserir como lista", "type": "string"}, "empresa": {"title": "Empresa", "description": "Nome da empresa que tal pessoa trabalha. Se tiver mais de uma, inserir como lista", "type": "string"}}, "required": ["pergunta", "nome", "empresa"]}
```


In [31]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate(
    template="Retorne corretamente o JSON baseado nas informações abaixo:\n{format_instructions}\n{sentenca}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": json_parser.get_format_instructions()},
)

print(prompt.format(sentenca="Steve jobs fundou a Apple, onde ele teve muito sucesso"))

Retorne corretamente o JSON baseado nas informações abaixo:
The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"pergunta": {"title": "Pergunta", "description": "Senten\u00e7a do usu\u00e1rio", "type": "string"}, "nome": {"title": "Nome", "description": "Nome de alguma pessoa citada na senten\u00e7a. Se tiver mais nomes, inserir como lista", "type": "string"}, "empresa": {"title": "Empresa", "description": "Nome da empresa que tal pessoa trabalha. Se tiver mais de uma, inserir como lista", "type": "string"}}, "required": ["pergunta", "nome", "empresa"]}
```
Steve jobs fundou a A

In [32]:
chain = prompt | llm | json_parser

In [34]:
resp_llm = chain.invoke({
    "sentenca": "Steve jobs fundou a Apple, onde ele teve muito sucesso"
})

In [35]:
print(resp_llm)

{'pergunta': 'Steve jobs fundou a Apple, onde ele teve muito sucesso', 'nome': 'Steve Jobs', 'empresa': 'Apple'}


## Pandas Parser

Parser interessante de utilizar quando você quer dar instruções em linguagem natural e devolver a resposta em um dataframe do pandas.

Aqui utilizaremos o arquivo movies.csv que contém informações sobre filmes.

In [40]:
import pandas as pd
from langchain.output_parsers import PandasDataFrameOutputParser

path_file = "../../dataset/movies.csv"
df_imdb = pd.read_csv(path_file)
df_imdb = df_imdb[
    [
    'Title', 
    'Rating', 
    'Year',
    'Budget', 
    'Income'
    ]
].copy()

In [44]:
df_imdb.head(2)

,Title,Rating,Year,Budget,Income
0,Avatar: The Way of Water,7.8,2022,"$350,000,000","$2,267,946,983"
1,Guillermo del Toro's Pinocchio,7.6,2022,"$35,000,000","$108,967"


In [48]:
df = pd.DataFrame(
    {
        "num_legs": [2, 4, 8, 0],
        "num_wings": [2, 0, 0, 0],
        "num_specimen_seen": [10, 2, 1, 8],
    }
)

In [49]:
df

,num_legs,num_wings,num_specimen_seen
0,2,2,10
1,4,0,2
2,8,0,1
3,0,0,8


In [53]:
# Set up a parser + inject instructions into the prompt template.
parser = PandasDataFrameOutputParser(dataframe=df)
print(parser)

ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [ ]:
from langchain.prompts import PromptTemplate

## Pydantic Parser

In [57]:
from typing import List

from langchain.output_parsers import PydanticOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field

class Ator(BaseModel):
    name: str = Field(description="Nome de um ator")
    lista_filme: List[str] = Field(description="Lista de filmes onde este ator atuou")

pydantic_parser = PydanticOutputParser(pydantic_object=Ator)
print(pydantic_parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"name": {"title": "Name", "description": "Nome de um ator", "type": "string"}, "lista_filme": {"title": "Lista Filme", "description": "Lista de filmes onde este ator atuou", "type": "array", "items": {"type": "string"}}}, "required": ["name", "lista_filme"]}
```


In [64]:
from langchain.prompts import PromptTemplate
prompt = PromptTemplate(
    template="Dado o nome do ator abaixo, liste 5 filmes onde esse ator atuou.\n{format_instructions}\n{ator}\n",
    input_variables=["ator"],
    partial_variables={"format_instructions": pydantic_parser.get_format_instructions()},
)

print(prompt.format(ator="Leonardo DiCaprio"))

Dado o nome do ator abaixo, liste 5 filmes onde esse ator atuou.
The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"name": {"title": "Name", "description": "Nome de um ator", "type": "string"}, "lista_filme": {"title": "Lista Filme", "description": "Lista de filmes onde este ator atuou", "type": "array", "items": {"type": "string"}}}, "required": ["name", "lista_filme"]}
```
Leonardo DiCaprio



In [65]:
chain = prompt | llm | pydantic_parser

In [66]:
resp_llm = chain.invoke({
    "ator": "Leonardo DiCaprio"
})

In [67]:
print(resp_llm)

name='Leonardo DiCaprio' lista_filme=['Titanic', 'O Lobo de Wall Street', 'Django Livre', 'A Origem', 'Era Uma Vez em... Hollywood']


## Structured Output Parser

Assim como o Pydantic Parser e o Json Parser, este aqui é utilizado para modelos mais simples.

In [86]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser

response_schemas = [
    ResponseSchema(name="pergunta", description="Pergunta do usuário"),
    ResponseSchema(name="resposta", description="Resposta que o modelo severá fornecer.",
    ),
]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
print(output_parser.get_format_instructions())

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"pergunta": string  // Pergunta do usuário
	"resposta": string  // Resposta que o modelo severá fornecer.
}
```


In [88]:
from langchain.prompts import PromptTemplate

template = """
Você irá ajudar a responder a pergunta abaixo da maneira mais correta possível.

{format_instructions}


{pergunta}
"""

prompt = PromptTemplate(
    template=template,
    input_variables=["pergunta"],
    partial_variables={"format_instructions": output_parser.get_format_instructions()}
)

print(prompt.format(pergunta="Quem foi Ronaldinho Gaúcho?"))


Você irá ajudar a responder a pergunta abaixo da maneira mais correta possível.

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"pergunta": string  // Pergunta do usuário
	"resposta": string  // Resposta que o modelo severá fornecer.
}
```


Quem foi Ronaldinho Gaúcho?



In [89]:
chain = prompt | llm | output_parser

In [90]:
resp_llm = chain.invoke({
    "pergunta": "Quem foi Ronaldinho Gaúcho?"
})

In [91]:
print(resp_llm)

{'pergunta': 'Quem foi Ronaldinho Gaúcho?', 'resposta': 'Ronaldinho Gaúcho, cujo nome completo é Ronaldo de Assis Moreira, é um ex-jogador de futebol brasileiro amplamente considerado um dos maiores jogadores de todos os tempos. Ele era conhecido por sua habilidade excepcional, dribles, passes e chutes poderosos.'}
